In [1]:
# ============================================================
# 🔮 INFÉRENCE – TF-IDF + ResNet50 + XGBoost (version corrigée)
# ============================================================

import os, re, html, joblib
import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import xgboost as xgb
from bs4 import BeautifulSoup
from scipy import sparse

# ============================================================
# 1️⃣ Chargement des artefacts
# ============================================================
print("📦 Chargement des modèles...")

# 🟢 Chemin correct pour ton vectorizer
vectorizer = joblib.load("data/processed/tfidf_vectorizer.joblib")
encoder    = joblib.load("data/models/label_encoder.joblib")
bst        = xgb.Booster()
bst.load_model("data/models/xgb_fusion.json")

print("✅ Artefacts chargés avec succès.")

# ============================================================
# 2️⃣ Fonction de nettoyage texte
# ============================================================
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text(" ")
    text = html.unescape(text)
    text = re.sub(r"[^A-Za-zÀ-ÖØ-öø-ÿ0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip().lower()
    return text

# ============================================================
# 3️⃣ Charger une ligne du CSV
# ============================================================
df = pd.read_csv("data/X_train_update.csv")
sample = df.sample(1, random_state=42).iloc[0]

designation = str(sample["designation"])
description = str(sample["description"])
imageid     = str(sample["imageid"])

print(f"\n🧾 Produit testé : {designation[:80]}...")

# ============================================================
# 4️⃣ TF-IDF texte
# ============================================================
text_clean = clean_text(designation + " " + description)
X_text = vectorizer.transform([text_clean])

# ============================================================
# 5️⃣ Embeddings image (ResNet50 PyTorch)
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet.fc = torch.nn.Identity()
resnet.eval().to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

img_path = f"data/images/images/image_train/{imageid}.jpg"
try:
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        X_img = resnet(x).cpu().numpy()
except Exception:
    print("⚠️ Image introuvable → vecteur nul utilisé.")
    X_img = np.zeros((1, 2048), dtype=np.float32)

# ============================================================
# 6️⃣ Fusion texte + image
# ============================================================
X_all = sparse.hstack([X_text, sparse.csr_matrix(X_img)]).tocsr()

# ============================================================
# 7️⃣ Prédiction XGBoost
# ============================================================
dtest = xgb.DMatrix(X_all)
proba = bst.predict(dtest)[0]
pred_id = np.argmax(proba)
prdtypecode = encoder.inverse_transform([pred_id])[0]

print(f"\n🎯 Code produit prédit : {prdtypecode}")

# ============================================================
# 8️⃣ Top-5 prédictions + catégories
# ============================================================
top5_idx = np.argsort(proba)[::-1][:5]
top5_codes = encoder.inverse_transform(top5_idx)
top5_scores = proba[top5_idx]

cat_map = {
    10:  "Livres et ouvrages culturels",
    40:  "Jeux vidéo et accessoires",
    50:  "Accessoires gaming",
    60:  "Consoles rétro",
    1140:"Figurines Pop & licences geek",
    1160:"Cartes à collectionner",
    1180:"Jeux de figurines & wargames",
    1280:"Jouets enfants & bébés",
    1281:"Jeux et loisirs enfants",
    1300:"Drones et modèles réduits",
    1301:"Chaussettes & accessoires enfants",
    1302:"Jouets divers & loisirs créatifs",
    1320:"Puériculture & équipement bébé",
    1560:"Mobilier & articles de maison",
    1920:"Linge de maison & décoration textile",
    1940:"Alimentation & boissons",
    2060:"Décoration & accessoires saisonniers",
    2220:"Accessoires pour animaux",
    2280:"Magazines & journaux anciens",
    2403:"Livres, mangas & partitions",
    2462:"Lots jeux vidéo et consoles",
    2522:"Fournitures de papeterie",
    2582:"Mobilier et accessoires de jardin",
    2583:"Accessoires pour piscines et spas",
    2585:"Outils et équipements de jardinage",
    2705:"Essais & livres d’histoire",
    2905:"Jeux PC à télécharger & éditions spéciales",
}

print("\n🏆 Top-5 prédictions :")
for c, s in zip(top5_codes, top5_scores):
    name = cat_map.get(int(c), "Non défini")
    print(f"  {c:<6} | {s:.3f} | {name}")

if int(prdtypecode) in cat_map:
    print(f"\n🪄 Catégorie prédite : {cat_map[int(prdtypecode)]}")
else:
    print("\n⚠️ Catégorie non référencée dans le mapping local.")


📦 Chargement des modèles...


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/tfidf_vectorizer.joblib'